# TBD Header

## Smoothing

Smoothing can help to discover trends that otherwise might be hard to see in raw data. 

In [40]:
%use lets-plot(apiVersion=0.0.21-SNAPSHOT)
%use krangl

In [41]:
val mpg_df = DataFrame.readCSV("https://jetbrains.bintray.com/lets-plot/mpg.csv")
mpg_df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


### The default smoothing method is `'linear model'` (or `'lm'`)

In [42]:
val dat = (mpg_df.names.filter {it.isNotBlank()}.map { Pair(it, mpg_df.get(it).values())}).toMap()

In [43]:
val mpg_plot = lets_plot(dat) {x="displ"; y="hwy"} 
mpg_plot + geom_point() + geom_smooth()

### `LOESS` model does seem to better fit MPG data than linear model.

In [44]:
mpg_plot + geom_point() + geom_smooth(method="loess", size=1.0)

### Applying smoothing to groups

Let's map the vehicle `drivetrain type` (variable 'drv') to the color of points.

This makes it easy to see that points with the same type of the drivetrain are forming some kind of groups or clusters. 

In [45]:
mpg_plot + geom_point {color="drv"} +
           geom_smooth(method="loess", size=1.0) {color="drv"}

### Apply linear model with 2nd degree polynomial.

As `LOESS` prediction looks a bit weird let's try 2nd degree polinomial regression.

In [46]:
mpg_plot + geom_point {color="drv"} +
           geom_smooth(method="lm", deg=2, size=1.0) {color="drv"}

## Using `as_discrete()` function with numeric data series

In the previous examples we were using a discrete (or categorical) variable `'drv'` to split the data into a groups.

Now let's try to use a numeric variable `'cyl'` for the same purpose.

In [47]:
mpg_plot + geom_point {color="cyl"} +
           geom_smooth(method="lm", deg=2, size=1.0) {color="cyl"}

Easy to see that the data wasn't split into groups. 
`Lets-Plot` offers two solutions in this situation:

 * Use the `group` aesthetic
 * Use the `as_discrete()` function

The **group** aesthetic helps to create a groups.

In [48]:
mpg_plot + geom_point {color="cyl"} +
           geom_smooth(method="lm", deg=2, size=1.0) {color="cyl"; group="cyl"}

The **as_discrete('cyl')** function will "annotate" the `'cyl'` variable as `discrete`.

This leads to creation of the groups and to assigning of a `discrete` color scale instead of a `continuous`.

In [49]:
// ToDo: implement as_discrete

// from lets_plot.mapping import as_discrete

// mpg_plot + geom_point(aes(color='cyl'))\
//          + geom_smooth(aes(color=as_discrete('cyl')), method='lm', deg=2, size=1)

## Effect of `span` parameter on the "wiggliness" the LOESS smoother.

The span is the fraction of points used to fit each local regression.
Small numbers make a wigglier curve, larger numbers make a smoother curve.

In [50]:
%use numpy(v=0.1.4)

import jetbrains.letsPlot.label.ggtitle
import kotlin.math.PI

import org.jetbrains.numkt.core.*
import org.jetbrains.numkt.math.*
import org.jetbrains.numkt.random.Random
import org.jetbrains.numkt.*

In [51]:
val n = 150
val x_range = arange<Double>(-2 * PI, 2 * PI, 4 * PI / n)
val y_range = sin(x_range) + array<Double>( List(n) { Random.uniform(-0.5, 0.5) } )
val df = mapOf(
    "x" to x_range.toList(),
    "y" to y_range.toList()
)

In [53]:
val p = ggplot(df) {x="x"; y="y"} + geom_point(shape=21, fill="yellow", color="#8c564b")
val p1 = p + geom_smooth(method="loess", size=1.5, color="#d62728") + ggtitle("default (span = 0.5)")
val p2 = p + geom_smooth(method="loess", span=.2, size=1.5, color="#9467bd") + ggtitle("span = 0.2")
val p3 = p + geom_smooth(method="loess", span=.7, size=1.5, color="#1f77b4") + ggtitle("span = 0.7")
val p4 = p + geom_smooth(method="loess", span=1, size=1.5, color="#2ca02c") + ggtitle("span = 1")

GGBunch()
.addPlot(p1, 0, 0, 400, 300)
.addPlot(p2, 400, 0, 400, 300)
.addPlot(p3, 0, 300, 400, 300)
.addPlot(p4, 400, 300, 400, 300)